## Data preparation

In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np 
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from deep_translator import GoogleTranslator
import ast

In [2]:
#Read in csv file with scraped Tweets into pandas dataframe
df = pd.read_csv(r'SGE_collected',header=None)

In [3]:
#Discover dataframe 
#=> missing column names
#=> some characters included that can be removed in the columns with indexes 1 and 2 for example
df.head()

,0,1,2,3
0,2023-02-08 19:21:35+00:00,"b'@EduardoSGE @sge_ge habe, die meinen Magen/D...",b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'"
1,2023-02-08 19:21:12+00:00,b'Eintracht hat das Derby gewonnen und die Sta...,b'SGEagle',"b'Nordrhein-Westfalen, Deutschla'"
2,2023-02-08 19:20:38+00:00,b'@iamdardann @lysander_sge 40er war echt brut...,b'sge_94',b''
3,2023-02-08 19:20:24+00:00,b'@EduardoSGE @sge_ge Ach das soll diese Sorge...,b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'"
4,2023-02-08 19:19:38+00:00,"b'R.I.P. Friedel Lutz, hier 3/2018 mit Dieter ...",b'fisPeppi',b'Frankfurt a.M.'


In [4]:
#Get length of dataframe
len(df)

14549

In [5]:
#Rename columns of the dataframe
df.columns = ["Date", "Tweet", "User", "Location"]

In [6]:
#Decode tweet column to utf-8
df['tweet'] = df['Tweet'].apply(ast.literal_eval).str.decode("utf-8")

In [7]:
#Create datetime column time from column Date
df['time'] = pd.to_datetime(df['Date'])

In [8]:
#Create Date column Date from datetime column time
df['Date'] = pd.to_datetime(df['time']).dt.date

In [9]:
#Get weekday, Hour and Minute columns
df['weekday'] = df['time'].dt.dayofweek
df['Hour'] = df['time'].dt.hour
df['Minute'] = df['time'].dt.minute

In [10]:
#Dataframe now looks like this
df.head()

,Date,Tweet,User,Location,tweet,time,weekday,Hour,Minute
0,2023-02-08,"b'@EduardoSGE @sge_ge habe, die meinen Magen/D...",b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'","@EduardoSGE @sge_ge habe, die meinen Magen/Dar...",2023-02-08 19:21:35+00:00,2,19,21
1,2023-02-08,b'Eintracht hat das Derby gewonnen und die Sta...,b'SGEagle',"b'Nordrhein-Westfalen, Deutschla'",Eintracht hat das Derby gewonnen und die Stadt...,2023-02-08 19:21:12+00:00,2,19,21
2,2023-02-08,b'@iamdardann @lysander_sge 40er war echt brut...,b'sge_94',b'',@iamdardann @lysander_sge 40er war echt brutal...,2023-02-08 19:20:38+00:00,2,19,20
3,2023-02-08,b'@EduardoSGE @sge_ge Ach das soll diese Sorge...,b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'",@EduardoSGE @sge_ge Ach das soll diese Sorge s...,2023-02-08 19:20:24+00:00,2,19,20
4,2023-02-08,"b'R.I.P. Friedel Lutz, hier 3/2018 mit Dieter ...",b'fisPeppi',b'Frankfurt a.M.',"R.I.P. Friedel Lutz, hier 3/2018 mit Dieter St...",2023-02-08 19:19:38+00:00,2,19,19


In [11]:
#Just take the results from 6th to 8th of February
df = df.loc[:'2023-02-05']
df

,Date,Tweet,User,Location,tweet,time,weekday,Hour,Minute
0,2023-02-08,"b'@EduardoSGE @sge_ge habe, die meinen Magen/D...",b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'","@EduardoSGE @sge_ge habe, die meinen Magen/Dar...",2023-02-08 19:21:35+00:00,2,19,21
1,2023-02-08,b'Eintracht hat das Derby gewonnen und die Sta...,b'SGEagle',"b'Nordrhein-Westfalen, Deutschla'",Eintracht hat das Derby gewonnen und die Stadt...,2023-02-08 19:21:12+00:00,2,19,21
2,2023-02-08,b'@iamdardann @lysander_sge 40er war echt brut...,b'sge_94',b'',@iamdardann @lysander_sge 40er war echt brutal...,2023-02-08 19:20:38+00:00,2,19,20
3,2023-02-08,b'@EduardoSGE @sge_ge Ach das soll diese Sorge...,b'Anna_scxl',"b'Manchmal Hessen, oft Hannover'",@EduardoSGE @sge_ge Ach das soll diese Sorge s...,2023-02-08 19:20:24+00:00,2,19,20
4,2023-02-08,"b'R.I.P. Friedel Lutz, hier 3/2018 mit Dieter ...",b'fisPeppi',b'Frankfurt a.M.',"R.I.P. Friedel Lutz, hier 3/2018 mit Dieter St...",2023-02-08 19:19:38+00:00,2,19,19
...,...,...,...,...,...,...,...,...,...
2019,2023-02-06,b'@SGE_Melli Ja schon aber trotzdem f\xc3\xbch...,b'Natascha_SGE',b'',@SGE_Melli Ja schon aber trotzdem fühlt man si...,2023-02-06 19:37:13+00:00,0,19,37
2020,2023-02-06,b'@sensinho82 @SGEintracht1899 Musst du ehrlic...,b'EduardoSGE',b'Im Herzen Von Europa ',@sensinho82 @SGEintracht1899 Musst du ehrlich ...,2023-02-06 19:35:37+00:00,0,19,35
2021,2023-02-06,b'@Natascha_SGE Klingt gut :)',b'taunusabbel',"b'Frankfurt am Main, Deutschland'",@Natascha_SGE Klingt gut :),2023-02-06 19:35:04+00:00,0,19,35
2022,2023-02-06,b'@faznet Wenn er sich hinter dem Pr\xc3\xa4si...,b'Schietweer',b'',@faznet Wenn er sich hinter dem Präsidenten st...,2023-02-06 19:34:59+00:00,0,19,34


In [12]:
#Unique values for column User in the dataframe
df['User'].unique()

array(["b'Anna_scxl'", "b'SGEagle'", "b'sge_94'", "b'fisPeppi'",
       "b'wetten_com'", "b'EduardoSGE'", "b'iamdardann'", "b'Netti_SGE'",
       "b'hangithigher365'", "b'lysander_sge'", "b'taunusabbel'",
       "b'DonMai79'", "b'EintrachtNeun'", "b'eintracht_lz'",
       "b'paaullSGE'", "b'Erinmund'", "b'trashpanda069'",
       "b'fabian_future'", "b'sge_bot'", "b'SGE_Melli'", "b'SGE4EVERde'",
       "b'Krey_CSM'", "b'Metalkruemel'", "b'SGE_Papa'", "b'sge_ge'",
       "b'SasDin1899'", "b'devilffm'", "b'hannes_hhs'", "b'sgestrala'",
       "b'Lukas_SGE_'", "b'GastornisSGE'", "b'Paashy_'", "b'Elijela97'",
       "b'GuyCalledGreezy'", "b'SGE_Elias'", "b'Track3eck'",
       "b'beves_welt'", "b'jonasbhl'", "b'Natascha_SGE'", "b'nurdie_sge'",
       "b'1tr8Fan'", "b'tim24572551'", "b'SorenWagner'", "b'timbruenjes'",
       "b'PascallllSGE'", "b'SGE_Henry'", "b'KevJadonCharles'",
       "b'gnutiez'", "b'SGE_HesseBub'", "b'oliver_sge'",
       "b'OleSteinberg84'", "b'EintrachtFrauen'", "b'ins

In [13]:
#Clean the text and username using functions in order to remove some specific characters

def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9\_]+','',text) #Removes @ mentions
    text = re.sub(r'_[A-Za-z0-9\_]+','',text)
    text = re.sub(r'#','', text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text) # Remove the hyper link
    return text

def cleanUsername(name):
    name = re.sub(r"b\'",'',name)
    name = re.sub(r"\'",'',name)
    return name

In [14]:
#Apply the clean Text function to the column tweet
df['tweet']=df['tweet'].apply(cleanTxt)

In [15]:
#Apply the cleanUsername function to the column User
df['User']=df['User'].apply(cleanUsername)

In [61]:
#Translate the tweets to english using GoogleTranslator in order to execute the Sentiment analysis in the next step
df['translated_tweet'] = df['tweet'].apply(lambda x: GoogleTranslator(source='de', target='en').translate(x))

In [62]:
#Initialize the english stopwords in order to avoid bias in the next steps where the tweets are analyzed
english_stopwords = stopwords.words('english')
#Initialize the CountVectorizer
vect = CountVectorizer(stop_words = english_stopwords)

In [63]:
#Remove all stopwords from all the translated tweets
df['tweet_without_stopwords'] = df['translated_tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (english_stopwords)]))

In [76]:
#Save dataframe as csv file
df.to_csv('SGE_prepared.csv')